# Code 2-7 to Code 2-10

# Evaluating rules

The rules file contains 2 sets of rules.
1. One with single word being present. If it is present sentence gets assigned to that category. For example the word “baggage” is present it gets assigned to the category. 
2. The other type of rule is an “And” of 2 words separated by a window. Based on general practice it is found that wide variety of topics could be coved by single word or a two word match .For example the rule - “check” and “in” separated (window) by less than 2 words would be able to capture a sentence like “check me in”. The column “window” in the rules file with values -1 means window is not applicable. Also note that a “hit” with any rule is sufficient for a sentence to get mapped. 
From that perspective all the rules are “Or”ed with each other

In [9]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score

##importing data and rules dataset

df = pd.read_csv("airline_dataset.csv",encoding ='latin1')
rules = pd.read_csv("rules_airline.csv")

def rule_eval(sent,word1,word2,class1,type1,window):
    class_fnl=""
    if(type1=="single"):
        if(sent.find(word1)>=0):
            class_fnl=class1
        else:
            class_fnl==""
    elif(type1=="double"):
        ## both words present
        if((sent.find(word1)>=0) &(sent.find(word2)>=0)):
            if(window==-1):
                class_fnl==class1
            else:
               ## both words present with window size 
                find_text = word1+ ".*" + word2
                list1 =  re.findall(find_text,sent)
                for i in list1:
                    window_size = i.count(' ')-1
                    
                    if(window_size<=window):
                        class_fnl = class1
                        break;
                
    else:
        class_fnl=""
    
    
    return class_fnl

The function rule_eval takes the sentence and the parameters of the rules and evaluates if there is “hit”. If there is a “hit” it assigns corresponding class to the sentence. For rule type “double” a regular expression of the form “word1.* word2” is used. This is to calculate the number of words between 2 matched words. If that number is below the window specified then its considered as a hit.

In [10]:
rules.head()

,word,word2,class,type,window
0,check in,none,check in,single,-1
1,checkin,none,check in,single,-1
2,checking in,none,check in,single,-1
3,bag,none,baggage,single,-1
4,luggage,none,baggage,single,-1


In [11]:
df["line"]=df["line"].str.lower().str.lstrip().str.rstrip()

In [12]:
###eval for each sentence
topics_list = []
for i,row in df.iterrows():
    sent = row["line"]
    for j,row1 in rules.iterrows():
        word1 = row1["word"]
        word2 = row1["word2"]
        class1 = row1["class"]
        type1 = row1["type"]
        window = row1["window"]
        class1 = rule_eval(sent,word1,word2,class1,type1,window)
        
        if(class1!=""):
            break;
            
    topics_list.append(class1)


In [13]:
df["topics"] = topics_list
df.loc[df.topics=="","topics"]="other"

In [14]:
df.head()

,line,class,topics
0,when can i web check-in?,check in,check in
1,want to check in,check in,check in
2,please check me in,check in,check in
3,check in,check in,check in
4,my flight is tomm can i check in,check in,check in


# Calculate Accuracy score

Once we have the assigned topics from the rules added to the dataset - we can compare the manual labelled calss and what was extracted from rules. We shall now measure the accuracy of our exercise by comparing manuall labels (class) to the topics assigned (topics) by rules. We do this by using the accuracy method in scikit learn

In [15]:
accuracy_score(df["class"], df["topics"], normalize=True, sample_weight=None)

0.7814070351758794

In [16]:
###print errors and modify rules
df[(df["class"]!=df.topics)]

,line,class,topics
25,how much can i carry,baggage,other
29,upto what weight i can carry,baggage,other
30,how much can i carry,baggage,other
60,error while logging in,login,greetings
64,log me in,login,other
65,log in but error,login,other
66,access account online,login,other
67,forgot username,login,other
68,not remembering username,login,other
72,what is username,login,other
